In [ ]:
# from the segmented image:
# the color channel r,g,b = color histogram
# mean, mode, std, skewness, energy, entropy, kurtosis
# min and max values for each channel.
# get the hsv space color info and calculate the same mean, ... for it too.
# glcm - contrast, correlation, enegry, homogenity

